In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

# Load your data
data = pd.read_csv('Results_21Mar2022.csv')

# Normalize the data across the entire dataset for parallel coordinates
impact_columns = {
    'mean_ghgs': 'Greenhouse Gas Emissions (kg)',
    'mean_land': 'Agricultural Land Use (sq m)',
    'mean_watscar': 'Water Scarcity',
    'mean_eut': 'Eutrophication Potential (gPO4e)',
    'mean_ghgs_ch4': 'Methane Emissions (kg)',
    'mean_ghgs_n2o': 'Nitrous Oxide Emissions',
    'mean_bio': 'Biodiversity Impact (species extinction per day)',
    'mean_watuse': 'Water Usage (cubic meters)',
    'mean_acid': 'Acidification Potential'
}
data2=data.copy()
#Normalise
for col in impact_columns.keys():
    data2[col] = (data2[col] - data2[col].min()) / (data2[col].max() - data2[col].min())

# Define full diet labels for readability
full_diet_labels = {
    'vegan': 'Vegans',
    'veggie': 'Vegetarians',
    'fish': 'Fish-eaters',
    'meat50': 'Low meat-eaters (<50 g d^-1)',
    'meat': 'Medium meat-eaters (50-99 g d^-1)',
    'meat100': 'High meat-eaters (≥100 g d^-1)'
}

colorscale = [
    [0.00, '#31a354'],  # Vegans
    [0.20, '#4daf4a'],  # Vegetarians
    [0.40, '#6baed6'],  # Fish-eaters
    [0.60, '#ffcc5c'],  # Low meat-eaters
    [0.80, '#fd8d3c'],  # Medium meat-eaters
    [1.00, '#843c39']   # High meat-eaters
]

# Diet group colors
diet_colors = {
    'Vegans': '#31a354',
    'Vegetarians': '#4daf4a',
    'Low meat-eaters (<50 g d^-1)': '#ffcc5c',
    'Medium meat-eaters (50-99 g d^-1)': '#fd8d3c',
    'High meat-eaters (≥100 g d^-1)': '#843c39',
    'Fish-eaters': '#6baed6'
}

diet_color_map = {
    'vegan': 1,
    'veggie': 2,
    'fish': 3,
    'meat50': 4,
    'meat': 5,
    'meat100': 6
}
# Define styles for the regular and active (selected) tabs
tab_style = {
    'padding': '6px 12px',  # Reduce padding to make the tab thinner
    'fontWeight': 'bold',
    'border': '1px solid #d6d6d6',
    'backgroundColor': '#f8f8f8',
    'borderRadius': '5px 5px 0 0',  # Rounded corners for the top
    'marginRight': '2px',  # Space between tabs
}

tab_selected_style = {
    'padding': '6px 12px',
    'fontWeight': 'bold',
    'border': '1px solid #aaa',
    'backgroundColor': '#fff',
    'color': 'black',
    'borderBottom': '1px solid #fff',  # Hide the bottom border to merge with the content area
}


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H2("Environmental Impact of Diet Types\nDashboard", style={
    'text-align': 'center', 
    'font-family': 'Segoe UI, sans-serif', 
    'font-weight': '500'
}),
html.Div([
    html.Span(style={
        'background-color': '#31a354', 
        'color': 'white', 
        'padding': '5px', 
        'margin-right': '10px', 
        'font-family': 'Segoe UI, sans-serif'
    }, children='Vegans'),
    html.Span(style={
        'background-color': '#4daf4a', 
        'color': 'white', 
        'padding': '5px', 
        'margin-right': '10px', 
        'font-family': 'Segoe UI, sans-serif'
    }, children='Vegetarians'),
    html.Span(style={
        'background-color': '#ffcc5c', 
        'color': 'black', 
        'padding': '5px', 
        'margin-right': '10px', 
        'font-family': 'Segoe UI, sans-serif'
    }, children='Low meat-eaters (<50 g d^-1)'),
    html.Span(style={
        'background-color': '#fd8d3c', 
        'color': 'white', 
        'padding': '5px', 
        'margin-right': '10px', 
        'font-family': 'Segoe UI, sans-serif'
    }, children='Medium meat-eaters (50-99 g d^-1)'),
    html.Span(style={
        'background-color': '#843c39', 
        'color': 'white', 
        'padding': '5px', 
        'margin-right': '10px', 
        'font-family': 'Segoe UI, sans-serif'
    }, children='High meat-eaters (≥100 g d^-1)'),
    html.Span(style={
        'background-color': '#6baed6', 
        'color': 'white', 
        'padding': '5px', 
        'font-family': 'Segoe UI, sans-serif'
    }, children='Fish-eaters'),
], style={
    'text-align': 'center', 
    'margin': '10px 0', 
    'font-size': '16px', 
    'font-family': 'Segoe UI, sans-serif'
}),
    dcc.Tabs([
        dcc.Tab(label='Sankey Diagram', children=[
            html.Div([
                html.P("Select an environmental impact factor for Sankey Diagram:"),
                dcc.Dropdown(
                    id='impact-factor-sankey',
                    options=[{'label': impact_columns[col], 'value': col} for col in impact_columns],
                    value='mean_ghgs'
                ),
                dcc.Graph(id='sankey-diagram')
            ])
        ],style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Parallel Coordinates Plot', children=[
            html.Div([
                html.P("Select environmental impacts for Parallel Coordinates Plot:"),
                dcc.Dropdown(
                    id='environmental-impacts-parcoords',
                    options=[{'label': v, 'value': k} for k, v in impact_columns.items()],
                    value=list(impact_columns.keys())[:4],
                    multi=True
                ),
                html.Div([
    html.Div([
        html.P("Enable Comparative Analysis (Min max Normalised) and Static Scaling:", style={'display': 'inline-block', 'margin-right': '10px'}),
        dcc.Checklist(
            id='comparative-scaling',
            options=[{'label': 'Enable', 'value': 'enabled'}],
            value=[],
            style={'display': 'inline-block', 'margin': '0'}
        )
    ], style={'display': 'inline-block', 'margin-right': '20px'}),  # Add margin-right for spacing between the controls

    html.Div([
        html.P("Average Across All Monte Carlo Runs:", style={'display': 'inline-block', 'margin-right': '10px'}),
        dcc.Checklist(
            id='average-runs',
            options=[{'label': 'Enable', 'value': 'average'}],
            value=[],
            style={'display': 'inline-block', 'margin': '0'}
        )
    ], style={'display': 'inline-block'}),  # Inline-block for side-by-side layout
], style={'width': '100%'}),html.Div([
                dcc.Graph(id='parallel-coordinates-plot')
            ], style={'padding': '0px 100px'})])
        ],style=tab_style, selected_style=tab_selected_style)
    ]),
    html.P("Select Diet Type:"),
    dcc.Dropdown(
        id='diet-type',
        options=[{'label': full_diet_labels.get(k, k), 'value': k} for k in full_diet_labels],
        value=list(full_diet_labels.keys()),
        multi=True
    ),html.Div([
    html.Div([
        html.P("Select Age Range:", style={'display': 'inline-block', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='age-group',
            options=[{'label': age, 'value': age} for age in ['All'] + sorted(data['age_group'].unique().tolist())],
            value='All',
            style={'display': 'inline-block', 'width': '300px'}
        ),
    ], style={'display': 'inline-block', 'margin-right': '20px'}),  # Ensure margin between the two controls
    html.Div([
        html.P("Select Gender:", style={'display': 'inline-block', 'margin-right': '10px'}),
        dcc.RadioItems(
            id='gender-filter',
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'Male', 'value': 'male'},
                {'label': 'Female', 'value': 'female'}
            ],
            value='All',
            style={'display': 'inline-block'}
        ),
    ], style={'display': 'inline-block'}),
], style={'width': '100%'}),
    html.Div([
        html.P("Choose between Arithmetic average of means vs Median of means:", style={'display': 'inline-block', 'margin-right': '10px'}),
        dcc.RadioItems(
            id='mean-median-switch',
            options=[
                {'label': 'Arithmetic Mean', 'value': 'mean'},
                {'label': 'Median', 'value': 'median'}
            ],
            value='mean',
            style={'display': 'inline-block'}
        ),
    ]),
])
 

# Callbacks for both Sankey and parallel coordinates plot
@app.callback(
    Output('sankey-diagram', 'figure'),
    [Input('impact-factor-sankey', 'value'), Input('diet-type', 'value'), Input('age-group', 'value'), Input('gender-filter', 'value'), Input('mean-median-switch', 'value')]
)
def update_sankey(selected_impact, selected_diet_types, selected_age_group, selected_gender, stat_method):
    filtered_data = data.copy()
    if selected_age_group != 'All':
        filtered_data = filtered_data[filtered_data['age_group'] == selected_age_group]
    if selected_gender != 'All':
        filtered_data = filtered_data[filtered_data['sex'] == selected_gender]
    if selected_diet_types:
        filtered_data = filtered_data[filtered_data['diet_group'].isin(selected_diet_types)]

    
    if stat_method == 'mean':
        grouped_data = filtered_data.groupby('diet_group')[selected_impact].mean().reset_index()
    else:  # 'median'
        grouped_data = filtered_data.groupby('diet_group')[selected_impact].median().reset_index()

    labels = [full_diet_labels.get(diet, diet) for diet in grouped_data['diet_group']] + ['Environmental Impact of ' + impact_columns[selected_impact]]
    source = [labels.index(full_diet_labels.get(diet, diet)) for diet in grouped_data['diet_group']]
    target = [labels.index(labels[-1]) for _ in grouped_data['diet_group']]
    value = list(grouped_data[selected_impact])
    node_colors = [diet_colors.get(full_diet_labels.get(diet, 'grey'), 'grey') for diet in grouped_data['diet_group']] + ['black']

    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color='black', width=0.25),
            label=labels,
            color=node_colors
        ),
        link=dict(
            source=source,
            target=target,
            value=value,
            color=['rgba' + str(tuple(int(diet_colors.get(full_diet_labels[diet], '#808080')[1:][i:i+2], 16) for i in (0, 2, 4)) + (0.5,)) for diet in grouped_data['diet_group']]
        )
    )])
    fig.update_layout(title_text='Sankey Diagram of ' + impact_columns[selected_impact], font_size=12)
    return fig

@app.callback(
    Output('parallel-coordinates-plot', 'figure'),
    [
        Input('diet-type', 'value'),
        Input('age-group', 'value'),
        Input('gender-filter', 'value'),
        Input('environmental-impacts-parcoords', 'value'),
        Input('mean-median-switch', 'value'),
        Input('average-runs', 'value'),
        Input('comparative-scaling', 'value')
    ]
)
def update_parallel_coordinates_plot(selected_diet_types, selected_age_group, selected_gender, selected_impacts, mean_or_median, average_runs, scaling_enabled):
    
    # Decide which dataset to use based on user input
    active_data = data2 if 'enabled' in scaling_enabled else data
    
    
    filtered_df = active_data.copy()
    if selected_age_group != 'All':
        filtered_df = filtered_df[filtered_df['age_group'] == selected_age_group]
    if selected_gender != 'All':
        filtered_df = filtered_df[filtered_df['sex'] == selected_gender]
    if selected_diet_types:
        filtered_df = filtered_df[filtered_df['diet_group'].isin(selected_diet_types)]

    if 'average' in average_runs:
        if mean_or_median == 'mean':
            filtered_df = filtered_df.groupby(['diet_group', 'age_group', 'sex'])[selected_impacts].mean().reset_index()
        else:  # 'median'
            filtered_df = filtered_df.groupby(['diet_group', 'age_group', 'sex'])[selected_impacts].median().reset_index()
#dimensions = [{'label': impact_columns[col], 'values': filtered_df[col], 'tickfont': dict(size=14)} for col in selected_impacts]
    if 'enabled' in scaling_enabled:
        dimensions = [{'label': impact_columns[col], 'values': filtered_df[col], 'range': [0, 1]} for col in selected_impacts]
    else:

        dimensions = [{'label': impact_columns[col], 'values': filtered_df[col],} for col in selected_impacts]

    
    
    # Dynamically generate the colorscale based on selected diet types
    new_colorscale = []
    for i, diet in enumerate(selected_diet_types):
        color = diet_colors[full_diet_labels[diet]]
        position = i / len(selected_diet_types)
        new_colorscale.append([position, color])
    
    # Ensure the last element is set to 1 for the end of the scale
    new_colorscale.append([1.0, diet_colors[full_diet_labels[selected_diet_types[-1]]]])

    fig = go.Figure(data=go.Parcoords(
        line=dict(
            color=[diet_color_map[diet] for diet in filtered_df['diet_group']],
            colorscale=new_colorscale,  # Use the dynamically generated colorscale
        ),
        dimensions=dimensions
    ))


    fig.update_layout(
         margin=dict(l=107),
        font=dict(family="Arial", size=18, color="Navy"),
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
